### 1 Hand-Crafted Network

### 3 Programming a neural network

In [22]:
import numpy as np
from sklearn import datasets

In [23]:
class ReLULayer(object):
    def forward(self, input):
        # remember the input for later backpropagation
        self.input = input
        # return the ReLU of the input
        relu = np.maximum(0, input) # ReLU as discussed in the lecture
        return relu

    def backward(self, upstream_gradient):
        # compute the derivative of ReLU from upstream_gradient and the stored input
        downstream_gradient = upstream_gradient * (self.input > 0) # derivative of ReLU of x > 0 is 1, otherwise 0
        return downstream_gradient

    def update(self, learning_rate):
        pass 

In [24]:
class OutputLayer(object):
    def __init__(self, n_classes):
        self.n_classes = n_classes

    def forward(self, input):
        # remember the input for later backpropagation
        self.input = input
        # return the softmax of the input
        softmax = np.exp(input) / np.sum(np.exp(input), axis=1, keepdims=True) # softmax as discussed in the lecture
        return softmax

    def backward(self, predicted_posteriors, true_labels):
        # return the loss derivative with respect to the stored inputs
        # (use cross-entropy loss and the chain rule for softmax,
        #  as derived in the lecture)
        # predicted_posteriors: the output of the softmax
        # true_labels: the true labels
        # eye matrix is used to get the one-hot encoding of the true labels
        downstream_gradient = (predicted_posteriors - np.eye(self.n_classes)[true_labels]) / true_labels.shape[0]
        return downstream_gradient

    def update(self, learning_rate):
        pass # softmax is parameter-free

In [25]:
class LinearLayer(object):
    def __init__(self, n_inputs, n_outputs):
        self.n_inputs  = n_inputs
        self.n_outputs = n_outputs
        # randomly initialize weights and intercepts
        # size of B is n_inputs x n_outputs, there is one weight for each input-output pair
        self.B = np.random.normal(loc=0.0, scale=1.0, size=(n_inputs, n_outputs))  # weights
        # size of b is 1 x n_outputs, there is one bias for each output
        self.b = np.random.normal(loc=0.0, scale=1.0, size=(n_outputs,))           # biases

    def forward(self, input):
        # remember the input for later backpropagation
        self.input = input
        # compute the scalar product of input and weights
        # (these are the preactivations for the subsequent non-linear layer)
        # and add the bias
        preactivations = np.dot(input, self.B) + self.b
        return preactivations

    def backward(self, upstream_gradient):
        # compute the derivative of the weights from
        # upstream_gradient and the stored input
        self.grad_b = np.sum(upstream_gradient, axis=0, keepdims=True)
        
        self.grad_B = np.dot(self.input.T, upstream_gradient)
        # compute the downstream gradient to be passed to the preceding layer
        downstream_gradient = np.dot(upstream_gradient, self.B.T) # your code here
        return downstream_gradient

    def update(self, learning_rate):
        # update the weights by batch gradient descent
        self.B = self.B - learning_rate * self.grad_B
        self.b = self.b - learning_rate * self.grad_b

In [26]:
class MLP(object):
    def __init__(self, n_features, layer_sizes):
        # constuct a multi-layer perceptron
        # with ReLU activation in the hidden layers and softmax output
        # (i.e. it predicts the posterior probability of a classification problem)
        #
        # n_features: number of inputs
        # len(layer_size): number of layers
        # layer_size[k]: number of neurons in layer k
        # (specifically: layer_sizes[-1] is the number of classes)
        self.n_layers = len(layer_sizes)
        self.layers   = []

        # create interior layers (linear + ReLU)
        n_in = n_features
        for n_out in layer_sizes[:-1]:
            self.layers.append(LinearLayer(n_in, n_out))
            self.layers.append(ReLULayer())
            n_in = n_out

        # create last linear layer + output layer
        n_out = layer_sizes[-1]
        self.layers.append(LinearLayer(n_in, n_out))
        self.layers.append(OutputLayer(n_out))

    def forward(self, X):
        # X is a mini-batch of instances
        batch_size = X.shape[0]
        # flatten the other dimensions of X (in case instances are images)
        X = X.reshape(batch_size, -1)

        # compute the forward pass
        # (implicitly stores internal activations for later backpropagation)
        result = X
        for layer in self.layers:
            result = layer.forward(result)
        return result

    def backward(self, predicted_posteriors, true_classes):
        # perform backpropagation w.r.t. the prediction for the latest mini-batch X
        downstream_gradient = self.layers[-1].backward(predicted_posteriors, true_classes)
        for layer in reversed(self.layers[:-1]):
            downstream_gradient = layer.backward(downstream_gradient)

    def update(self, X, Y, learning_rate):
        posteriors = self.forward(X)
        self.backward(posteriors, Y)
        for layer in self.layers:
            layer.update(learning_rate)

    def train(self, x, y, n_epochs, batch_size, learning_rate):
        N = len(x)
        n_batches = N // batch_size
        for i in range(n_epochs):
            # print("Epoch", i)
            # reorder data for every epoch
            # (i.e. sample mini-batches without replacement)
            permutation = np.random.permutation(N)

            for batch in range(n_batches):
                # create mini-batch
                start = batch * batch_size
                x_batch = x[permutation[start:start+batch_size]]
                y_batch = y[permutation[start:start+batch_size]]

                # perform one forward and backward pass and update network parameters
                self.update(x_batch, y_batch, learning_rate)

In [27]:
def make_moons(n_features, n_classes):
    network_2 = MLP(n_features, [2, 2, n_classes])
    network_3 = MLP(n_features, [3, 3, n_classes])
    network_5 = MLP(n_features, [5, 5, n_classes])
    network_30 = MLP(n_features, [30, 30, n_classes])
    return [network_2, network_3, network_5, network_30]

In [28]:
# set training/test set size
N = 2000

# create training and test data
X_train, Y_train = datasets.make_moons(N, noise=0.05)
X_test,  Y_test  = datasets.make_moons(N, noise=0.05)
n_features = 2
n_classes  = 2

# standardize features to be in [-1, 1]
offset  = X_train.min(axis=0)
scaling = X_train.max(axis=0) - offset
X_train = ((X_train - offset) / scaling - 0.5) * 2.0
X_test  = ((X_test  - offset) / scaling - 0.5) * 2.0

# set hyperparameters (play with these!)
layer_sizes = [5, 5, n_classes]
n_epochs = 5
batch_size = 200
learning_rate = 0.05

# create network
networks = make_moons(n_features, n_classes)

for network in networks:
    # train
    network.train(X_train, Y_train, n_epochs, batch_size, learning_rate)
    # test
    predicted_posteriors = network.forward(X_test)
    # determine class predictions from posteriors by winner-takes-all rule
    predicted_classes = np.argmax(predicted_posteriors, axis=1) # your code here
    # compute and output the error rate of predicted_classes
    error_rate = np.mean(predicted_classes != Y_test) # your code here
    print("error rate:", error_rate)

error rate: 0.1535
error rate: 0.4305
error rate: 0.161
error rate: 0.005
